In [80]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2
import numpy as np
import glob

In [81]:
batch_size = 16
num_classes = 5
img_height = 50
img_width = 50

In [82]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '../../../data/fifty_fifty/',
  validation_split=0.2,
    color_mode="grayscale",
    label_mode='int',
  subset="training",
    shuffle=True,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '../../../data/fifty_fifty/',
  validation_split=0.2,
  color_mode="grayscale",
    label_mode='int',
    shuffle=True,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)

Found 29232 files belonging to 5 classes.
Using 23386 files for training.
Found 29232 files belonging to 5 classes.
Using 5846 files for validation.
['center', 'down', 'left', 'right', 'up']


In [72]:
input_layer = keras.Input(shape=(img_height, img_width, 1))
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(input_layer)
x = keras.layers.Conv2D(32, 1, padding='same', activation='relu')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Conv2D(64, 1, padding='same', activation='relu')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Conv2D(128, 1, padding='same', activation='relu')(x)
x = keras.layers.MaxPooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
output_layer = keras.layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs=input_layer, outputs=output_layer, name="jupiter3")

opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
# opt = tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9)
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
model.compile(optimizer=opt, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
   
model.summary()

Model: "jupiter3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 50, 50, 1)]       0         
_________________________________________________________________
rescaling_3 (Rescaling)      (None, 50, 50, 1)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 50, 50, 32)        64        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 25, 25, 64)        2112      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 12, 12, 64)        0  

In [73]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = tf.keras.callbacks.ModelCheckpoint('./best_model_aug.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
callbacks = [es,mc]

In [74]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=50, 
  batch_size =batch_size,
    callbacks=callbacks
)

Epoch 1/50
1424/1424 [==============================] - ETA: 0s - loss: 1.1276 - accuracy: 0.7777
Epoch 00001: val_accuracy improved from -inf to 0.89621, saving model to ./best_model_aug.h5
1424/1424 [==============================] - 48s 34ms/step - loss: 1.1276 - accuracy: 0.7777 - val_loss: 1.0115 - val_accuracy: 0.8962
Epoch 2/50
1423/1424 [============================>.] - ETA: 0s - loss: 1.0079 - accuracy: 0.8984
Epoch 00002: val_accuracy improved from 0.89621 to 0.92290, saving model to ./best_model_aug.h5
1424/1424 [==============================] - 49s 35ms/step - loss: 1.0079 - accuracy: 0.8984 - val_loss: 0.9835 - val_accuracy: 0.9229
Epoch 3/50
1422/1424 [============================>.] - ETA: 0s - loss: 0.9793 - accuracy: 0.9257
Epoch 00003: val_accuracy improved from 0.92290 to 0.93361, saving model to ./best_model_aug.h5
1424/1424 [==============================] - 49s 34ms/step - loss: 0.9794 - accuracy: 0.9257 - val_loss: 0.9726 - val_accuracy: 0.9336
Epoch 4/50
1422/

In [12]:
# model.save('../models/jupiter4')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../models/jupiter4/assets


In [ ]:
def add_augmentation(images:list):
    for idx, img in enumerate(images):
        aug_img = tf.image.random_brightness(img,max_delta=0.2)
        if (aug_img.mean() > 200) or (aug_img.mean() < 60):
            continue
        else:
            images[idx] = aug_img
    return images

In [78]:
def augment(path, brightness:bool, contrast:bool):
    files = glob.glob(path + '/*.jpg')
    count = 1
    for file in files:
        img = cv2.imread(file, cv2.IMREAD_UNCHANGED)
        if brightness == True:
            img = tf.image.random_brightness(img,max_delta=0.2)
        if contrast == True:
            img = tf.image.random_contrast(img, 0.2, 0.5)
        img = np.array(img)
        if (img.mean() > 210) or (img.mean() < 50):
            continue
        cv2.imwrite(os.path.join(path,f'aug{count}.jpg'),img)
        count+=1

In [79]:
for path in ['../../../data/fifty_fifty/center/',
             '../../../data/fifty_fifty/up/',
             '../../../data/fifty_fifty/down/',
             '../../../data/fifty_fifty/right/',
             '../../../data/fifty_fifty/left/']:
    augment(path, brightness=True, contrast=False)

In [76]:
augment('../../../data/fifty_fifty3/center/', brightness=False, contrast=True)

InvalidArgumentError: input must be at least 3-D, got shape[50,50] [Op:AdjustContrastv2]

In [61]:
img = cv2.imread('../../../data/fifty_fifty/center/aug2125.jpg', cv2.IMREAD_UNCHANGED)
img2 = cv2.imread('../../../data/fifty_fifty/center/aug2148.jpg', cv2.IMREAD_UNCHANGED)
img3 = cv2.imread('../../../data/fifty_fifty/center/aug2179.jpg', cv2.IMREAD_UNCHANGED)

In [2]:
model = tf.keras.models.load_model('./Models/jupiter1')

In [55]:
img1 = cv2.imread('../data/eye_imgs/right/a1.jpg' ,cv2.IMREAD_UNCHANGED)
img2 = cv2.imread('../data/eye_imgs/left/a10.jpg' ,cv2.IMREAD_UNCHANGED)
# img = img.reshape((100,100,1))
# img = np.expand_dims(img,0)

In [56]:
predict_both(left = img1, right = img2)

3

In [24]:
def joint_prediction():
    pass

In [42]:
img1 = np.expand_dims(img1,0)
img2 = np.expand_dims(img2,0)

In [43]:
a = np.concatenate((img1,img2))

In [27]:
a.shape

(2, 100, 100)

In [45]:
model.predict(a)

array([ 2.0141094, -0.907617 ,  3.9984462, -2.9328032,  1.9947188],
      dtype=float32)

In [53]:
def predict_both(left, right):
    img1 = np.expand_dims(left,0)
    img2 = np.expand_dims(right,0)
    concat = np.concatenate((img1,img2))
    classes = model.predict_classes(concat)
    if classes[0] == classes[1]:
        return classes[0]
    else:
        pred = model.predict(concat)
        a = (pred[0].argmax(), pred[0].max())
        b = (pred[1].argmax(), pred[1].max())
        if a[1] > b[1]:
            return a[0]
        else:
            return b[0]

In [ ]:
def predict_frame(self, eye_img):
    img = eye_img.copy()
    batch = np.expand_dims(img,0)
    prediction = self.model.predict_classes(batch)[0]
    return prediction

In [211]:
class_names

['center', 'down', 'left', 'right', 'up']

In [212]:
model.predict_classes(img)[0]

2